<a href="https://colab.research.google.com/github/austiezr/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [X] Continue to clean and explore your data. 
- [X] For the evaluation metric you chose, what score would you get just by guessing?
- [X] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [0]:
%%capture

!pip3 install category_encoders
!pip3 install pandas-profiling
import pandas_profiling
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder, OneHotEncoder, TargetEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_classif, mutual_info_regression, f_classif
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import RidgeClassifierCV, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.decomposition import PCA

## Load Data

In [0]:
df = pd.read_csv('/Users/austie/Downloads/student/student-mat.csv', sep=';')
df1 = pd.read_csv('/Users/austie/Downloads/student/student-por.csv', sep=';')
df = df.append(df1, ignore_index=True)

In [6]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


## Wrangle Data / Create Target / Split

In [0]:
%%capture

df['famsize'] = df['famsize'].map({'GT3': 1, 'LE3':0})
df['address'] = df['address'].map({'U':1, 'R':0})
df['Pstatus'] = df['Pstatus'].map({'A': 0, 'T':1})
for col in ['schoolsup', 'famsup', 'paid', 'activities', 
            'nursery', 'higher', 'internet', 'romantic']:
  df[col] = df[col].map({'yes':1, 'no':0})

for col in ['G1', 'G2', 'G3']:
  df[col] = df[col].replace({0: np.NaN})

dfmean = df[['G1', 'G2', 'G3']]
dfmean['Mean'] = dfmean.mean(axis=1)
df['Mean'] = dfmean['Mean']
df['Pass/Fail'] = df['G3']>=10

df['Improved'] = (df['G3']-df['Mean'])>0

df = df.drop(columns=['school'])

training, test = train_test_split(df, train_size=0.80, test_size=0.20, random_state=33)
train, val = train_test_split(training, train_size=0.80, test_size=0.20, random_state=33)

In [0]:
target = 'Improved'
features = train.drop(columns=[target, 'G1', 'G2', 'Mean']).columns
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]
X_training = training[features]
y_training = training[target]

## Majority Class Baseline

In [10]:
df[target].value_counts(normalize=True)

True     0.536398
False    0.463602
Name: Improved, dtype: float64

## Fast First Model

In [19]:
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    LogisticRegressionCV(n_jobs=-1,
                         random_state=33)
    )


pipe.fit(X_training, y_training)

print(f'Training Accuracy: {pipe.score(X_training, y_training)}\n')
print(f'Test Accuracy: {pipe.score(X_test, y_test)}')

Training Accuracy: 0.6526946107784432

Test Accuracy: 0.6507177033492823
